<a href="https://colab.research.google.com/github/hyo250/gittest/blob/master/%EB%B9%85%EB%B6%84%EA%B8%B0%EC%8B%A4%EA%B8%B0_%EC%9E%91%EC%97%85%ED%98%952_%EC%B5%9C%EC%A2%85_hyo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
import numpy as np
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/dataq/main/y_train.csv')
X_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/dataq/main/X_train.csv',encoding='euc-kr')
test  = pd.read_csv('https://raw.githubusercontent.com/Datamanim/dataq/main/X_test.csv',encoding='euc-kr')

In [22]:
#훈련데이터 만들기. (이거 나중에 train, val set 나눠서 모델 만들고 예측데이터 만들어서 정확도 확인해보고 test에 모델적용할것임.)
train=pd.merge(X_train,y_train)

In [23]:
train.head(10)

,cust_id,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기,gender
0,0,68282840,11264000,6860000.0,기타,강남점,19,3.894737,0.527027,17,0
1,1,2136000,2136000,300000.0,스포츠,잠실점,2,1.500000,0.000000,1,0
2,2,3197000,1639000,NaN,남성 캐주얼,관악점,2,2.000000,0.000000,1,1
3,3,16077620,4935000,NaN,기타,광주점,18,2.444444,0.318182,16,1
4,4,29050000,24000000,NaN,보석,본 점,2,1.500000,0.000000,85,0
5,5,11379000,9552000,462000.0,디자이너,일산점,3,1.666667,0.200000,42,0
6,6,10056000,7612000,4582000.0,시티웨어,강남점,5,2.400000,0.333333,42,0
7,7,514570080,27104000,29524000.0,명품,본 점,63,2.634921,0.222892,5,0
8,8,688243360,173088000,NaN,기타,본 점,18,5.944444,0.411215,15,0
9,9,26640850,13728000,NaN,농산물,대전점,1,12.000000,0.000000,0,1


# 전처리(결측치 제거/대체,  이상치 확인 후 변경)

In [24]:
train.isnull().sum()

cust_id       0
총구매액          0
최대구매액         0
환불금액       2295
주구매상품         0
주구매지점         0
내점일수          0
내점당구매건수       0
주말방문비율        0
구매주기          0
gender        0
dtype: int64

In [25]:
test.isnull().sum()

cust_id       0
총구매액          0
최대구매액         0
환불금액       1611
주구매상품         0
주구매지점         0
내점일수          0
내점당구매건수       0
주말방문비율        0
구매주기          0
dtype: int64

In [26]:
#환불금액에 nan은 환불안한거겠지 그니까 0으로 채우자!
train['환불금액']=train['환불금액'].fillna(0)
test['환불금액']=test['환불금액'].fillna(0)

In [27]:
#데이터 이상치 있는지 기술분석
train.describe()

,cust_id,총구매액,최대구매액,환불금액,내점일수,내점당구매건수,주말방문비율,구매주기,gender
count,3500.000000,3.500000e+03,3.500000e+03,3.500000e+03,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000
mean,1749.500000,9.191925e+07,1.966424e+07,8.289786e+06,19.253714,2.834963,0.307246,20.958286,0.376000
std,1010.507298,1.635065e+08,3.199235e+07,3.010204e+07,27.174942,1.912368,0.289752,24.748682,0.484449
min,0.000000,-5.242152e+07,-2.992000e+06,0.000000e+00,1.000000,1.000000,0.000000,0.000000,0.000000
25%,874.750000,4.747050e+06,2.875000e+06,0.000000e+00,2.000000,1.666667,0.027291,4.000000,0.000000
50%,1749.500000,2.822270e+07,9.837000e+06,0.000000e+00,8.000000,2.333333,0.256410,13.000000,0.000000
75%,2624.250000,1.065079e+08,2.296250e+07,2.642250e+06,25.000000,3.375000,0.448980,28.000000,1.000000
max,3499.000000,2.323180e+09,7.066290e+08,5.637530e+08,285.000000,22.083333,1.000000,166.000000,1.000000


총구매액에 이상한 음수값이 있다.. 제거요망

In [28]:
print(train['최대구매액'].min()); print(train['총구매액'].min());
print(test['최대구매액'].min()); print(test['총구매액'].min());

-2992000
-52421520
-37440000
-37440000


In [37]:
#총구매액보다 더 높은 최대구매액이 있다?!
이상값={'index': [], '총구매액': [], '최대구매액':[]}
cnt=0
for (i,j) in zip(train.총구매액, train.최대구매액):
  if i< j:
      이상값['index'].append(cnt)
      이상값['총구매액'].append(i)
      이상값['최대구매액'].append(j)
  cnt+=1

이상값_df=pd.DataFrame(이상값)
이상값_df

,index,총구매액,최대구매액
0,61,1062500,2125000
1,138,208000,954000
2,167,3132000,3628800
3,188,10321200,12222000
4,254,4748600,12375000
...,...,...,...
64,3198,53661280,130870000
65,3346,9369060,11792000
66,3379,1300000,2967000
67,3411,0,3520000


In [39]:
3500-69

3431

In [41]:
# 이 이상값이 든 인덱스들을 다 없애야겠음.
train=train.drop(이상값_df['index'], axis=0)
train.shape # 3500-69인 값이 행으로 남음!

(3431, 11)

In [42]:
train.shape

(3431, 11)

In [43]:
#다시 이상한 값이 있는지 보자
이상값={'index': [], '총구매액': [], '최대구매액':[]}
cnt=0
for (i,j) in zip(train.총구매액, train.최대구매액):
  if i< j:
      이상값['index'].append(cnt)
      이상값['총구매액'].append(i)
      이상값['최대구매액'].append(j)
  cnt+=1

이상값_b=pd.DataFrame(이상값)
print(이상값_b); print(이상값_b.shape)


Empty DataFrame
Columns: [index, 총구매액, 최대구매액]
Index: []
(0, 3)


- 이상치 삭제 성공! 
- 이제 train셋을 x 데이터와 y 데이터로 분리해보자. 

In [44]:
train.describe()

,cust_id,총구매액,최대구매액,환불금액,내점일수,내점당구매건수,주말방문비율,구매주기,gender
count,3431.000000,3.431000e+03,3.431000e+03,3.431000e+03,3431.000000,3431.000000,3431.000000,3431.000000,3431.000000
mean,1750.950743,9.354521e+07,1.961213e+07,7.686850e+06,19.473331,2.845892,0.305773,21.010201,0.378898
std,1011.005505,1.646780e+08,3.185452e+07,2.671794e+07,27.333964,1.919946,0.290252,24.810328,0.485183
min,0.000000,-2.992000e+06,-2.992000e+06,0.000000e+00,1.000000,1.000000,0.000000,0.000000,0.000000
25%,877.500000,5.050175e+06,2.855600e+06,0.000000e+00,2.000000,1.687500,0.020771,4.000000,0.000000
50%,1750.000000,2.914895e+07,9.900000e+06,0.000000e+00,9.000000,2.340000,0.252577,13.000000,0.000000
75%,2627.500000,1.095116e+08,2.316080e+07,2.180000e+06,26.000000,3.383730,0.445718,28.000000,1.000000
max,3499.000000,2.323180e+09,7.066290e+08,3.815210e+08,285.000000,22.083333,1.000000,166.000000,1.000000


In [45]:
X=train.drop(['cust_id', 'gender'], axis=1)
y=train['gender']

# train_test_split하고 모델적용하기 전에! 
- 범주형변수들을 원핫인코딩하고, 스케일링 할것..

In [50]:
X_dum=pd.get_dummies(X)
test_dum=pd.get_dummies(test)

In [51]:
X_dum

,총구매액,최대구매액,환불금액,내점일수,내점당구매건수,주말방문비율,구매주기,주구매상품_가공식품,주구매상품_가구,주구매상품_건강식품,주구매상품_골프,주구매상품_구두,주구매상품_기타,주구매상품_남성 캐주얼,주구매상품_남성 트랜디,주구매상품_남성정장,주구매상품_농산물,주구매상품_대형가전,주구매상품_디자이너,주구매상품_란제리/내의,주구매상품_명품,주구매상품_모피/피혁,주구매상품_보석,주구매상품_생활잡화,주구매상품_섬유잡화,주구매상품_셔츠,주구매상품_소형가전,주구매상품_수산품,주구매상품_스포츠,주구매상품_시티웨어,주구매상품_식기,주구매상품_아동,주구매상품_악기,주구매상품_액세서리,주구매상품_육류,주구매상품_일용잡화,주구매상품_젓갈/반찬,주구매상품_주류,주구매상품_주방가전,주구매상품_주방용품,주구매상품_차/커피,주구매상품_축산가공,주구매상품_침구/수예,주구매상품_캐주얼,주구매상품_커리어,주구매상품_통신/컴퓨터,주구매상품_트래디셔널,주구매상품_피혁잡화,주구매상품_화장품,주구매지점_강남점,주구매지점_관악점,주구매지점_광주점,주구매지점_노원점,주구매지점_대구점,주구매지점_대전점,주구매지점_동래점,주구매지점_미아점,주구매지점_본 점,주구매지점_부산본점,주구매지점_부평점,주구매지점_분당점,주구매지점_상인점,주구매지점_센텀시티점,주구매지점_안양점,주구매지점_영등포점,주구매지점_울산점,주구매지점_인천점,주구매지점_일산점,주구매지점_잠실점,주구매지점_전주점,주구매지점_창원점,주구매지점_청량리점,주구매지점_포항점
0,68282840,11264000,6860000.0,19,3.894737,0.527027,17,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2136000,2136000,300000.0,2,1.500000,0.000000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,3197000,1639000,0.0,2,2.000000,0.000000,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,16077620,4935000,0.0,18,2.444444,0.318182,16,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,29050000,24000000,0.0,2,1.500000,0.000000,85,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,3175200,3042900,0.0,1,2.000000,1.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3496,29628600,7200000,6049600.0,8,1.625000,0.461538,40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3497,75000,75000,0.0,1,1.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3498,1875000,1000000,0.0,2,1.000000,0.000000,39,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [52]:
test_dum

,cust_id,총구매액,최대구매액,환불금액,내점일수,내점당구매건수,주말방문비율,구매주기,주구매상품_가공식품,주구매상품_가구,주구매상품_건강식품,주구매상품_골프,주구매상품_구두,주구매상품_기타,주구매상품_남성 캐주얼,주구매상품_남성 트랜디,주구매상품_남성정장,주구매상품_농산물,주구매상품_대형가전,주구매상품_디자이너,주구매상품_란제리/내의,주구매상품_명품,주구매상품_모피/피혁,주구매상품_보석,주구매상품_생활잡화,주구매상품_섬유잡화,주구매상품_셔츠,주구매상품_수산품,주구매상품_스포츠,주구매상품_시티웨어,주구매상품_식기,주구매상품_아동,주구매상품_악기,주구매상품_액세서리,주구매상품_육류,주구매상품_일용잡화,주구매상품_젓갈/반찬,주구매상품_주류,주구매상품_주방가전,주구매상품_주방용품,주구매상품_차/커피,주구매상품_축산가공,주구매상품_침구/수예,주구매상품_캐주얼,주구매상품_커리어,주구매상품_통신/컴퓨터,주구매상품_트래디셔널,주구매상품_피혁잡화,주구매상품_화장품,주구매지점_강남점,주구매지점_관악점,주구매지점_광주점,주구매지점_노원점,주구매지점_대구점,주구매지점_대전점,주구매지점_동래점,주구매지점_미아점,주구매지점_본 점,주구매지점_부산본점,주구매지점_부평점,주구매지점_분당점,주구매지점_상인점,주구매지점_센텀시티점,주구매지점_안양점,주구매지점_영등포점,주구매지점_울산점,주구매지점_인천점,주구매지점_일산점,주구매지점_잠실점,주구매지점_전주점,주구매지점_창원점,주구매지점_청량리점,주구매지점_포항점
0,3500,70900400,22000000,4050000.0,13,1.461538,0.789474,26,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3501,310533100,38558000,48034700.0,90,2.433333,0.369863,3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,3502,305264140,14825000,30521000.0,101,14.623762,0.083277,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3503,7594080,5225000,0.0,5,2.000000,0.000000,47,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,3504,1795790,1411200,0.0,3,2.666667,0.125000,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2477,5977,82581500,23976000,0.0,8,1.750000,0.642857,40,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2478,5978,480000,480000,0.0,1,1.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2479,5979,260003790,25750000,0.0,19,3.736842,0.915493,18,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2480,5980,88991520,18120000,0.0,5,3.600000,0.444444,60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [53]:
test_dum=test_dum.drop(['cust_id'],axis=1)

In [54]:
print(X_dum.columns); print(len(X_dum.columns))
print(test_dum.columns) ;print(len(test_dum.columns))

Index(['총구매액', '최대구매액', '환불금액', '내점일수', '내점당구매건수', '주말방문비율', '구매주기',
       '주구매상품_가공식품', '주구매상품_가구', '주구매상품_건강식품', '주구매상품_골프', '주구매상품_구두',
       '주구매상품_기타', '주구매상품_남성 캐주얼', '주구매상품_남성 트랜디', '주구매상품_남성정장', '주구매상품_농산물',
       '주구매상품_대형가전', '주구매상품_디자이너', '주구매상품_란제리/내의', '주구매상품_명품', '주구매상품_모피/피혁',
       '주구매상품_보석', '주구매상품_생활잡화', '주구매상품_섬유잡화', '주구매상품_셔츠', '주구매상품_소형가전',
       '주구매상품_수산품', '주구매상품_스포츠', '주구매상품_시티웨어', '주구매상품_식기', '주구매상품_아동',
       '주구매상품_악기', '주구매상품_액세서리', '주구매상품_육류', '주구매상품_일용잡화', '주구매상품_젓갈/반찬',
       '주구매상품_주류', '주구매상품_주방가전', '주구매상품_주방용품', '주구매상품_차/커피', '주구매상품_축산가공',
       '주구매상품_침구/수예', '주구매상품_캐주얼', '주구매상품_커리어', '주구매상품_통신/컴퓨터', '주구매상품_트래디셔널',
       '주구매상품_피혁잡화', '주구매상품_화장품', '주구매지점_강남점', '주구매지점_관악점', '주구매지점_광주점',
       '주구매지점_노원점', '주구매지점_대구점', '주구매지점_대전점', '주구매지점_동래점', '주구매지점_미아점',
       '주구매지점_본  점', '주구매지점_부산본점', '주구매지점_부평점', '주구매지점_분당점', '주구매지점_상인점',
       '주구매지점_센텀시티점', '주구매지점_안양점', '주구매지점_영등포점', '주구매지점_울산점', '주구매지점_인천점',
       '주구매지점_일산점', '주구매지점_잠실점', '주구매지

In [55]:
#차이나는 컬럼 수 맞춰주자
remain_col=set(X_dum.columns)-set(test_dum.columns)
for c in remain_col:
  test_dum[c]=0
print(test_dum)

           총구매액     최대구매액        환불금액  ...  주구매지점_청량리점  주구매지점_포항점  주구매상품_소형가전
0      70900400  22000000   4050000.0  ...           0          0           0
1     310533100  38558000  48034700.0  ...           0          0           0
2     305264140  14825000  30521000.0  ...           0          0           0
3       7594080   5225000         0.0  ...           0          0           0
4       1795790   1411200         0.0  ...           1          0           0
...         ...       ...         ...  ...         ...        ...         ...
2477   82581500  23976000         0.0  ...           0          0           0
2478     480000    480000         0.0  ...           0          0           0
2479  260003790  25750000         0.0  ...           0          0           0
2480   88991520  18120000         0.0  ...           0          0           0
2481     623700    209000         0.0  ...           0          0           0

[2482 rows x 73 columns]


In [59]:
from sklearn.preprocessing import MinMaxScaler #범주형데이터 들어간 데이터와 수치형변수 들어간 거 한꺼번에 스켈링해도됨
sc=MinMaxScaler()
X_sc=sc.fit_transform(X_dum)
test_sc=sc.fit_transform(test_dum)

In [66]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val=train_test_split(X_sc, y, test_size=0.2, random_state=200, stratify=y)


In [67]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

#랜덤포레스트 모델
rf=RandomForestClassifier()
rf_model=rf.fit(X_train, y_train)
rf_pred=rf_model.predict(X_val)

#xgboost 모델
xgb=XGBClassifier(n_estimators=100)
xgb_model=xgb.fit(X_train,y_train)
xgb_pred=xgb_model.predict(X_val)

#정확도 비교
print('rf 정확도: ', accuracy_score(y_val, rf_pred))
print('xgb 정확도: ', accuracy_score(y_val, xgb_pred))

print('rf 정확도1: ',roc_auc_score(y_val, rf_pred))
print('xgb 정확도1: ', roc_auc_score(y_val, xgb_pred))



rf 정확도:  0.6506550218340611
xgb 정확도:  0.6724890829694323
rf 정확도1:  0.5850927760763827
xgb 정확도1:  0.6116825797153667


In [68]:
#xgboost 모델로 test셋에 적용시켜 값 만들어보자
result_pred=xgb_model.predict_proba(test_sc)
result_pred

array([[0.48230147, 0.5176985 ],
       [0.77553767, 0.22446233],
       [0.8488684 , 0.15113157],
       ...,
       [0.5198422 , 0.4801578 ],
       [0.56122386, 0.43877617],
       [0.591812  , 0.40818802]], dtype=float32)

In [78]:
np.shape(result_pred)

(2482, 2)

- result_pred는 왼쪽은 여자일 확률, 오른쪽은 남자일 확률을 나타냄...
- 여자일 확률 0.48230147, 남자일 확률 0.5176835....
- 남자데이터 기준으로 내라고 했으니까 오른쪽 값만 넣어줘야 함 (2482, 1) 형태로 다시 만들어야 함. 

In [82]:
result={'cust_id': [], 'gender': []}
result['cust_id']=test['cust_id']
result['gender']=pd.Series(result_pred[:, 1]) #1이 남자일 확률임
최종= pd.DataFrame(result)
최종

,cust_id,gender
0,3500,0.517699
1,3501,0.224462
2,3502,0.151132
3,3503,0.528331
4,3504,0.393016
...,...,...
2477,5977,0.517237
2478,5978,0.446939
2479,5979,0.480158
2480,5980,0.438776


In [85]:
최종.to_csv('수험번호1234.csv', index=False)

In [86]:
#w제출 확인
df=pd.read_csv('수험번호1234.csv')
df.head()

,cust_id,gender
0,3500,0.517698
1,3501,0.224462
2,3502,0.151132
3,3503,0.528331
4,3504,0.393016
